# Chapter 2

## Modules and database connection

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import sql_magic
%reload_ext sql_magic

Microsoft SQL Server connection

In [6]:
server = 'DESKTOP-90A96QN'
database = 'Sandbox'
username = 'DESKTOP-90A96QN\krzys'
password = ''

# If logging using windows authentication, use trusted_connection=yes. This ignores username and password.
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
%config SQL.conn_name = 'engine'

Connection test

In [3]:
%read_sql SELECT TOP 3 * FROM orders where customer_id = 'A'

Query started at 10:56:14 AM Central European Daylight Time; Query executed in 0.01 m

,customer_id,order_id,product,quantity
0,A,1,taco,10
1,A,1,burger,15
2,A,3,taco,12


## Notes

**Analysis steps**
- Exploration - what the data is about? What are the tables?
- Profiling - unique values, distribution, basic statistics (similar to EDA)
- Cleaning - errors, nulls
- Modelling - applying a structure
- Analysis

## Exercises

### Profiling - Distribution

#### Page 38
How many customers had how many orders? Creating dataset for a histogram.

In [11]:
query = '''

DROP TABLE IF EXISTS orders;

CREATE TABLE orders (
	customer_id VARCHAR(50),
	order_id VARCHAR(50),
	product VARCHAR(200),
	quantity INT
	);

INSERT INTO orders (customer_id, order_id, product, quantity)
VALUES	('A','1','taco',10),
		('A','1','burger',15),
		('B','2','taco',5),
		('A','3','taco',12),
		('B','4','burger',8),
		('B','4','salad',15),
		('B','4','tomato',20),
		('A','5','taco',5),
		('A','5','salad',8),
		('A','1','burger',15),
		('D','2','taco',5),
		('D','3','taco',12),
		('E','4','burger',8),
		('A','4','salad',15),
		('B','4','tomato',20),
		('E','5','taco',5),
		('A','5','salad',8),
		('B','4','tomato',20),
		('E','5','taco',5),
		('B','5','salad',8),
		('X','4','tomato',20),
		('X','5','taco',5),
		('X','5','salad',8);

SELECT * FROM orders;
'''

%read_sql {query}

Query started at 11:21:03 AM Central European Daylight Time; Query executed in 0.00 m

,customer_id,order_id,product,quantity
0,A,1,taco,10
1,A,1,burger,15
2,B,2,taco,5
3,A,3,taco,12
4,B,4,burger,8
5,B,4,salad,15
6,B,4,tomato,20
7,A,5,taco,5
8,A,5,salad,8
9,A,1,burger,15


In [12]:
query = '''
SELECT orders, COUNT(*) AS num_customers
FROM
(
	SELECT customer_id, COUNT(order_id) AS orders
	FROM orders
	GROUP BY customer_id
) AS tbl_1
GROUP BY orders;

'''

%read_sql {query}

Query started at 11:21:06 AM Central European Daylight Time; Query executed in 0.00 m

,orders,num_customers
0,2,1
1,3,2
2,7,1
3,8,1


#### Page 42
Binning using subqery and ntile.

In [19]:
query = '''

DROP TABLE IF EXISTS orders;

CREATE TABLE orders (
	customer_id VARCHAR(50),
	order_id VARCHAR(50),
	product VARCHAR(200),
	quantity INT,
    amount NUMERIC(30,2)
	);

INSERT INTO orders (customer_id, order_id, product, quantity, amount)
VALUES	('A','1','taco',10,19.99),
		('A','1','burger',15,9.99),
		('B','2','taco',5,59.99),
		('A','3','taco',12,11.99),
		('C','4','burger',8,23.49),
		('D','4','salad',15,55.98),
		('A','4','tomato',20,12.99),
		('B','5','taco',5,99.99),
		('C','5','salad',8,14.99),
		('D','1','burger',15,34.99),
		('E','2','taco',5,4.99),
		('F','3','taco',12,89.99);

SELECT * FROM orders ORDER BY 1
'''

%read_sql {query}

Query started at 11:22:44 AM Central European Daylight Time; Query executed in 0.00 m

,customer_id,order_id,product,quantity,amount
0,A,1,taco,10,19.99
1,A,1,burger,15,9.99
2,A,3,taco,12,11.99
3,A,4,tomato,20,12.99
4,B,5,taco,5,99.99
5,B,2,taco,5,59.99
6,C,4,burger,8,23.49
7,C,5,salad,8,14.99
8,D,1,burger,15,34.99
9,D,4,salad,15,55.98


Here are the bins:

In [31]:
query = '''
SELECT customer_id, order_id, amount,
       NTILE(10) OVER(ORDER BY amount) AS bins
FROM orders
'''

%read_sql {query}

Query started at 11:30:54 AM Central European Daylight Time; Query executed in 0.00 m

,customer_id,order_id,amount,bins
0,E,2,4.99,1
1,A,1,9.99,1
2,A,3,11.99,2
3,A,4,12.99,2
4,C,5,14.99,3
5,A,1,19.99,4
6,C,4,23.49,5
7,D,1,34.99,6
8,D,4,55.98,7
9,B,2,59.99,8


Find lower and upper bounds of each bin.

In [33]:
query = '''

SELECT bins,
       MIN(amount) AS lower_bound,
       MAX(amount) AS upper_bound,
       COUNT(order_id) AS order_id
FROM
(
    SELECT customer_id, order_id, amount,
           NTILE(10) OVER(ORDER BY amount) AS bins
    FROM orders
) AS tb1
GROUP BY bins
'''

%read_sql {query}

Query started at 11:31:16 AM Central European Daylight Time; Query executed in 0.00 m

,bins,lower_bound,upper_bound,order_id
0,1,4.99,9.99,2
1,2,11.99,12.99,2
2,3,14.99,14.99,1
3,4,19.99,19.99,1
4,5,23.49,23.49,1
5,6,34.99,34.99,1
6,7,55.98,55.98,1
7,8,59.99,59.99,1
8,9,89.99,89.99,1
9,10,99.99,99.99,1


### Profiling - Data Quality